In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import argparse, os, time
import urllib3, random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import winsound



In [2]:
df = pd.read_csv('first_v2_mod.csv', sep = ';')

In [3]:
# Convert 'date_connected' to date_time
dc = df['date_connected']
dc_right = []
for data in dc:
    dc_right.append( dt.strptime(data, '%Y-%m-%d'))

df['date_connected'] = dc_right
df.head()

,Unnamed: 0,name,link,occupation,location,mutual_raw,date_connected_string,see_connections_link,date_connected,city,region,country
0,0,Madhu Shurpali,/in/madhu-shurpali-2aabab14/,Project Manager at Tata Technologies,"Coventry, United Kingdom",10 Mutual Connections,"June 6, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-06-06,Coventry,NaN,United Kingdom
1,1,Ivan Lafuenti,/in/ivan-l-b7458932/,Design Engineering at Giannuzzi srl,"Brindisi Area, Italy","['2 Mutual Connections', 'You both studied at ...","May 30, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-30,Brindisi,NaN,Italy
2,2,Tiziano Ruggiero,/in/tiziano-ruggiero-2011b2164/,Responsabile del reparto tecnico di produzione...,"Avezzano, Abruzzo, Italy",Tiziano started a new position as Responsabile...,"May 30, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-30,Avezzano,Abruzzo,Italy
3,3,Antonio Guerrieri,/in/antonio-guerrieri/,Ingegnere at Università del Salento,"Lecce, Apulia, Italy",You both studied at Università del Salento,"May 29, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-29,Lecce,Apulia,Italy
4,4,Lorenzo Gagliardi,/in/lorenzo-gagliardi-72049b45/,Ing,"Savona, Liguria, Italy","['19 Mutual Connections', 'You both worked at ...","May 2, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-02,Savona,Liguria,Italy


# funcs

In [4]:
def open_my_page():
    login_user_cell_id = 'username'
    login_pw_cell_id = 'password'
    # launch browser
    browser = webdriver.Chrome()
    # open start page
    browser.get("https://www.linkedin.com/login")
    time.sleep(3)
    #
    my_user = 'ing.bianco.roberto@gmail.com'
    emailElement = browser.find_element_by_id( login_user_cell_id )
    emailElement.send_keys(my_user)
    time.sleep(.5)
    #
    my_pw = 'password01'
    pwElement = browser.find_element_by_id( login_pw_cell_id )
    pwElement.send_keys(my_pw)
    time.sleep(.5)
    # Submit form
    pwElement.submit()
    #
    return browser

In [5]:
def extract_tag_from_html(page, tag_string, parameter, parameter_value):
    # returns the tag where <tag_string parameter = 'parameter_value'>
    result = []
    tags = page.find_all(tag_string)
    for tag in tags:
        tag_parameter = tag.get(parameter)
        if tag_parameter:
            if parameter_value in tag_parameter:
                result.append(tag)
    return result

In [6]:
def between_tag_markers(page, tag_string, parameter, parameter_value):
    
    tag_list = extract_tag_from_html(page, tag_string, parameter, parameter_value)
    result = []
    for tag in tag_list:
        # transforms into a unique string and removes \n
        tag_string = re.sub('\n', '', str(tag))
        between_tag = re.findall('>(.*)<', tag_string)
        result.append( between_tag[0].strip() )
    return result

In [7]:
def within_tag_marker(page, tag_string, parameter, parameter_value, field_of_interest):
    # reports the field of interest within the tag
    tag_list = extract_tag_from_html(page, tag_string, parameter, parameter_value)
    result = []
    for tag in tag_list:
        out = tag.get(field_of_interest)
        result.append(out)
    return result

In [8]:
def scroll_to_bottom(browser):
    # scrolls to bottom to load/show entire page
    #browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
    time.sleep(random.uniform(.3,.5))
    lenOfPage = browser.execute_script("window.scrollBy(0, 800);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            #time.sleep(1)
            #browser.execute_script("window.scrollTo(0, document.body.scrollHeight/3);")
            time.sleep(random.uniform(.3,.5))
            lenOfPage = browser.execute_script("window.scrollBy(0, 800);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True

In [9]:
def find_next_button(browser):
    buttons_browser = browser.find_elements_by_tag_name('button')
    
    page = BeautifulSoup(browser.page_source)
    buttons_page = page.find_all('button')
    
    for k in range(len(buttons_page)):
        
        button_string = re.sub('\n', '', str(buttons_page[k]))
        between_tag = re.findall(\
                '<span class="artdeco-button__text">\s*Next<', button_string)
        if between_tag:
           return (buttons_browser[k], buttons_page[k])
    

In [10]:
def is_button_enabled(button):
    button_string = re.sub('\n', '', str(button))
    if re.findall('disabled', button_string):
        is_enabled = False
    else:
        is_enabled = True
    return is_enabled


In [11]:
def scan_second_page(page):
    names = []
    names+= between_tag_markers(page, 'span', 'class', 'actor-name')
    
    #links
    links = []
    link_group = within_tag_marker(page,'a','data-control-name','search_srp_result','href')
    links += link_group[0::2]
        
    # occupation
    qq = extract_tag_from_html(page, 'p','class','subline-level-1')

    occupations= []
    for k in range(len(qq)):
        ww =re.findall('<span dir="ltr">(.*)</span>', str(qq[k]))
        occupations+=(ww)
        
    # location
    qq = extract_tag_from_html(page, 'p','class','subline-level-2')

    locations= []
    for k in range(len(qq)):
        ww =re.findall('<span dir="ltr">(.*)</span>', str(qq[k]))
        locations+=(ww)
        
    return (names, links, occupations, locations)


In [12]:
def collect_contacts(browser):
    names = []
    occupations = []
    locations = []
    links = []
    #
    scroll_to_bottom(browser)
    #time.sleep(1)
    page = BeautifulSoup(browser.page_source)
    # scan first page
    (new_names, new_links, new_occupations, new_locations) = scan_second_page(page)
    #print(len(new_names))
    # attach res
    names+=new_names
    links += new_links
    occupations+= new_occupations
    locations += new_locations

    # get next button
    (next_button_browser, next_button_page) = find_next_button(browser)
    # check is enabled
    is_next_enabled = is_button_enabled(next_button_page)

    while is_next_enabled:
        
        next_button_browser.click()
        time.sleep(random.uniform(1,2))
        #
        scroll_to_bottom(browser)
        time.sleep(.5)
        page = BeautifulSoup(browser.page_source)
        # scan first page
        (new_names, new_links, new_occupations, new_locations) = scan_second_page(page)
        #print(len(new_names))
        # attach res
        names+=new_names
        links+=new_links
        occupations+= new_occupations
        locations += new_locations

        # get next button
        (next_button_browser, next_button_page) = find_next_button(browser)
        # check is enabled
        is_next_enabled = is_button_enabled(next_button_page)
        
    return (names, links, occupations, locations)


# Main script

In [13]:
browser = open_my_page()

In [179]:
# initialise
full_names = []
full_links = []
full_occupations = []
full_locations = []

for k in range(len(df)):
    full_names.append('')
    full_links.append('')
    full_occupations.append('')
    full_locations.append('')
    
    


In [185]:
root_link = 'https://linkedin.com'
# beep every group_size
group_size = 1
failed_runs = []

for k in range(10):
    # open connections 
    print('Page ' + str(k+1))
    
    # beep control
    rest = (k+1)%group_size
    if rest == 0:
        ncycle = (k+1)/group_size
        for k in np.arange(start=0, stop= ncycle):
            winsound.Beep(1000, 150)
        
    # get data
    browser.get(root_link + df['see_connections_link'][k])
    
    #try:
    (names, links, occupations, locations) = collect_contacts(browser)
    # append
    full_names[k]=names
    full_links[k] = links
    full_occupations[k]=occupations
    full_locations[k]= locations
    #except:
     #   print('Failed run: ' +str(k))
      #  failed_runs.append(k)
    
# final beep
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 1500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

Page 1


TypeError: cannot unpack non-iterable NoneType object

In [189]:
page = BeautifulSoup(browser.page_source)
(a, b, c, d) = scan_second_page(page)

In [190]:
a

["José Carlos D'Andrea", 'Roberto Bianco', 'Guilherme Eisinger']

# Write csv

In [194]:
import csv
with open('second_connections_K4.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=';')
    errs = 0
    for k in [4]:
        for q in range(len(full_names[k])):
            row = [k, full_names[k][q], full_links[k][q],\
                      full_occupations[k][q], full_locations[k][q]]
            try:
                
                csv_writer.writerow( row )
            except:
                errs +=1
                print([k,q])
                print(row)
                
print('Total errors: '+ str(errs))


IndexError: string index out of range

In [197]:
full_links[4]

''

# Second version (smaller)

In [14]:
# initialise
full_names = []
full_links = []
full_occupations = []
full_locations = []
num_res = []

for k in range(len(df)):
    full_names.append('')
    full_links.append('')
    full_occupations.append('')
    full_locations.append('')
    num_res.append('')

In [17]:
root_link = 'https://linkedin.com'
# beep every group_size
group_size = 20
failed_runs = []

for k in range(48,len(df)):
    # open connections 
    print('Page ' + str(k+1))
    

    # get data
    if type(df['see_connections_link'][k]) == str:
        
        browser.get(root_link + df['see_connections_link'][k])
        time.sleep(random.uniform(2, 3.5))
        page = BeautifulSoup(browser.page_source)
        (names, links, occupations, locations) = \
            scan_second_page(page)
        # append
        full_names[int(k)]=names
        full_links[int(k)] = links
        full_occupations[int(k)]=occupations
        full_locations[int(k)]= locations
        num_res[int(k)] = between_tag_markers\
            (page,'h3','class','search-results__total')[0]
        
    # beep control
    rest = (k+1)%group_size
    if rest == 0:
        ncycle = (k+1)/group_size
        for k in np.arange(start=0, stop= ncycle):
            winsound.Beep(1000, 150)
        
# final beep
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 1500  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

Page 49
Page 50
Page 51
Page 52
Page 53
Page 54
Page 55
Page 56
Page 57
Page 58
Page 59
Page 60
Page 61
Page 62
Page 63
Page 64
Page 65
Page 66
Page 67
Page 68
Page 69
Page 70
Page 71
Page 72
Page 73
Page 74
Page 75
Page 76
Page 77
Page 78
Page 79
Page 80
Page 81
Page 82
Page 83
Page 84
Page 85
Page 86
Page 87
Page 88
Page 89
Page 90
Page 91
Page 92
Page 93
Page 94
Page 95
Page 96
Page 97
Page 98
Page 99
Page 100
Page 101
Page 102
Page 103
Page 104
Page 105
Page 106
Page 107
Page 108
Page 109
Page 110
Page 111
Page 112
Page 113
Page 114
Page 115
Page 116
Page 117
Page 118
Page 119
Page 120
Page 121
Page 122
Page 123
Page 124
Page 125
Page 126
Page 127
Page 128
Page 129
Page 130
Page 131
Page 132
Page 133
Page 134
Page 135
Page 136
Page 137
Page 138
Page 139
Page 140
Page 141
Page 142
Page 143
Page 144
Page 145
Page 146
Page 147
Page 148
Page 149
Page 150
Page 151
Page 152
Page 153
Page 154
Page 155
Page 156
Page 157
Page 158
Page 159
Page 160
Page 161
Page 162
Page 163
Page 164
Page 16

In [21]:
browser.get(root_link + df['see_connections_link'][290])

TypeError: can only concatenate str (not "float") to str

In [18]:
import csv
with open('simple_second_connections_48_293.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=';')
    errs = 0
    for k in range(len(full_names)):
        for q in range(len(full_names[k])):
            row = [k, full_names[k][q], full_links[k][q],\
                      full_occupations[k][q], full_locations[k][q],\
                  num_res[k] ]
            try:
                csv_writer.writerow( row )
            except:
                errs +=1
                print([k,q])
                print(row)
                
print('Total errors: '+ str(errs))


[71, 2]
[71, 'Carl Woodward', '/in/engineeringdirectoruk/', '⭐️ Project Management | ⭐️ Engineering Leadership | ⭐️ Developing people = Delivering Performance.', 'Derby, United Kingdom', 'Showing 999 results']
[122, 0]
[122, 'Nigel Maris', '/in/nigel-maris-aa40b31/', 'MD at AES Ltd ► End-to-end electronic solutions provider from innovative design &amp; prototyping to full scale production', 'Coventry, United Kingdom', 'Showing 998 results']
[201, 1]
[201, 'Ferhat Oğultekin', '/in/ferhat-o%C4%9Fultekin-a82a1636/', 'Pricing Manager EMEA Aftermarket at NGK Spark Plug Europe GmbH', 'Cologne Area, Germany', 'Showing 995 results']
[205, 0]
[205, 'Gill Dixon', '/in/gill-dixon/', 'Director: Teachers To Your Home\u200b. Over 18,000 teachers across the UK who also choose to offer high quality home tuition.', 'Oxford, United Kingdom', 'Showing 224 results']
[205, 2]
[205, 'Jim Dixon', '/in/jim-dixon-/', 'Director: Teachers To Your Home\u200b. Over 18,000 teachers across the UK who also choose to 

In [22]:
num_48_293 = pd.DataFrame(num_res)
num_48_293

,0
0,
1,
2,
3,
4,
5,
6,
7,
8,
9,


In [237]:
df.head()

,Unnamed: 0,name,link,occupation,location,mutual_raw,date_connected_string,see_connections_link,date_connected,city,region,country,num_res
0,0,Madhu Shurpali,/in/madhu-shurpali-2aabab14/,Project Manager at Tata Technologies,"Coventry, United Kingdom",10 Mutual Connections,"June 6, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-06-06,Coventry,NaN,United Kingdom,Showing 47 results
1,1,Ivan Lafuenti,/in/ivan-l-b7458932/,Design Engineering at Giannuzzi srl,"Brindisi Area, Italy","['2 Mutual Connections', 'You both studied at ...","May 30, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-30,Brindisi,NaN,Italy,Showing 285 results
2,2,Tiziano Ruggiero,/in/tiziano-ruggiero-2011b2164/,Responsabile del reparto tecnico di produzione...,"Avezzano, Abruzzo, Italy",Tiziano started a new position as Responsabile...,"May 30, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-30,Avezzano,Abruzzo,Italy,Showing 29 results
3,3,Antonio Guerrieri,/in/antonio-guerrieri/,Ingegnere at Università del Salento,"Lecce, Apulia, Italy",You both studied at Università del Salento,"May 29, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-29,Lecce,Apulia,Italy,Showing 197 results
4,4,Lorenzo Gagliardi,/in/lorenzo-gagliardi-72049b45/,Ing,"Savona, Liguria, Italy","['19 Mutual Connections', 'You both worked at ...","May 2, 2019",/search/results/people/?facetConnectionOf=%5B%...,2019-05-02,Savona,Liguria,Italy,Showing 533 results


In [238]:
df.to_csv('First_with_num_res.csv',sep=';')

In [23]:
num_48_293.to_csv('First_num_res_48_293.csv',sep=';')